<a href="https://colab.research.google.com/github/bkkaggle/lm-experiments/blob/master/transformers_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

Sat Dec 21 18:09:21 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
DIST_BUCKET="gs://tpu-pytorch/wheels"
TORCH_WHEEL="torch-1.15-cp36-cp36m-linux_x86_64.whl"
TORCH_XLA_WHEEL="torch_xla-1.15-cp36-cp36m-linux_x86_64.whl"
TORCHVISION_WHEEL="torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl"

# Install Colab TPU compat PyTorch/TPU wheels and dependencies
!pip uninstall -y torch torchvision
!gsutil cp "$DIST_BUCKET/$TORCH_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCH_XLA_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCHVISION_WHEEL" .
!pip install "$TORCH_WHEEL"
!pip install "$TORCH_XLA_WHEEL"
!pip install "$TORCHVISION_WHEEL"
!sudo apt-get install libomp5

In [0]:
!pip install git+https://github.com/bkkaggle/transformers fire wandb line_profiler

import os

!git clone https://github.com/NVIDIA/apex
os.chdir('apex')
!pip install -v --no-cache-dir ./
os.chdir('../')

!git clone https://github.com/bkkaggle/lm-experiments.git
os.chdir('lm-experiments/')

!wandb login 08283aec9b92f676724219169ea3c28cf41c85cc

In [0]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
Unpacking objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
From https://github.com/bkkaggle/lm-experiments
   45e5273..a075bfa  master     -> origin/master
Updating 45e5273..a075bfa
Fast-forward
 finetune.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)


In [0]:
!python finetune.py --dataset_path ./data/imdb/imdb-ctrl-imdb.pkl --model_type ctrl --checkpoint ctrl --optimizer SGD --lr 1e-3 --batch_size 1 --epochs 1

Traceback (most recent call last):
  File "finetune.py", line 236, in <module>
    fire.Fire(main)
  File "/usr/local/lib/python3.6/dist-packages/fire/core.py", line 138, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/usr/local/lib/python3.6/dist-packages/fire/core.py", line 471, in _Fire
    target=component.__name__)
  File "/usr/local/lib/python3.6/dist-packages/fire/core.py", line 675, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "finetune.py", line 228, in main
    import torch_xla.core.xla_model as xm
ModuleNotFoundError: No module named 'torch_xla'


In [0]:
# !python run_pplm.py -B military --cond_text "The potato" --length 100 --gamma 1.5 --num_iterations 3 --num_samples 1 --stepsize 0.03 --window_length 5 --kl_scale 0.01 --gm_scale 0.99 --sample
!python run_pplm.py -D sentiment --class_label 2 --cond_text "The potato" --length 100 --gamma 1.5 --num_iterations 3 --num_samples 1 --stepsize 0.03 --window_length 5 --kl_scale 0.01 --gm_scale 0.99 --sample

In [0]:
!python run_pplm.py -B ./data/pplm/sample-bow.txt --cond_text "The potato" --length 100 --gamma 1.5 --num_iterations 3 --num_samples 1 --stepsize 0.03 --window_length 5 --kl_scale 0.01 --gm_scale 0.99 --sample

In [0]:
import pandas as pd
df = pd.DataFrame([['positive', 'text'], ['negative', 'text2']], columns=['label', 'text'])
df.to_csv('data.tsv', sep='\t', index=False)

In [0]:
!python run_pplm_discrim_train.py --dataset SST --pretrained_model distilgpt2 --epochs 1 --log_interval 1000 --cached --save_model
!python run_pplm_discrim_train.py --dataset generic --dataset_fp ./data.tsv --pretrained_model distilgpt2 --epochs 1 --log_interval 1000 --cached --save_model
!python run_pplm.py -D generic --discrim_weights ./SST_classifier_head_epoch_1.pt --discrim_meta ./SST_classifier_head_meta.json --class_label 2 --cond_text "The potato" --length 100 --gamma 1.5 --num_iterations 3 --num_samples 1 --stepsize 0.03 --window_length 5 --kl_scale 0.01 --gm_scale 0.99 --sample